In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, cross_validate, cross_val_predict
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, precision_recall_curve
from sklearn.preprocessing import RobustScaler, MinMaxScaler

In [2]:
file = "../nba_forecast/nba_forecast/data/player_ratio_scores.csv"
file_2 = "../nba_forecast/nba_forecast/data/last_ncaa_season.csv"
file_3 = "../nba_forecast/nba_forecast/data/dataset_X_cleaned.csv"
ratios = pd.read_csv(file)
features = pd.read_csv(file_2)
athletics = pd.read_csv(file_3)

In [3]:
ratios.head()

,player_id,player_name,pos,off_score,def_score,uni_off_score,uni_def_score,ratio_off,ratio_def
0,anthony-davis-5,Anthony Davis,PF,3.96,3.34,3.68,3.79,1.08,0.88
1,michael-kidd-gilchrist-1,Michael Kidd-Gilchrist,SF,2.68,1.89,2.73,1.96,0.98,0.96
2,bradley-beal-1,Bradley Beal,SG,2.73,1.71,2.93,1.99,0.93,0.86
3,dion-waiters-1,Dion Waiters,SG,2.36,1.49,2.97,2.04,0.79,0.73
4,thomas-robinson-2,Thomas Robinson,PF,2.68,2.17,3.03,2.67,0.88,0.81


In [4]:
features.head()

,player_name,season,school_name,conf_abbr,g,gs,mp,per,ts_pct,efg_pct,...,usg_pct,ows,dws,ws,ws_per_40,obpm,dbpm,bpm,years,player_id
0,Anthony Davis,2011-12,Kentucky,SEC,40,40,1281,35.1,0.654,0.628,...,18.8,5.9,4.1,9.9,0.310,9.1,8.1,17.2,1,anthony-davis-5
1,Michael Kidd-Gilchrist,2011-12,Kentucky,SEC,40,39,1245,21.2,0.570,0.511,...,20.6,3.4,2.6,6.0,0.192,4.1,3.6,7.7,1,michael-kidd-gilchrist-1
2,Bradley Beal,2011-12,Florida,SEC,37,37,1267,22.0,0.575,0.525,...,23.0,3.6,2.1,5.7,0.180,5.4,2.9,8.4,1,bradley-beal-1
3,Dion Waiters,2011-12,Syracuse,Big East,37,0,891,26.3,0.565,0.534,...,26.7,3.2,1.7,5.0,0.223,6.0,4.8,10.8,2,dion-waiters-1
4,Thomas Robinson,2011-12,Kansas,Big 12,39,39,1242,27.4,0.549,0.512,...,30.0,3.7,3.5,7.2,0.231,5.4,3.4,8.8,3,thomas-robinson-2


In [5]:
athletics = athletics.drop(athletics[athletics.player_name == 'Justin Jackson (UMD)'].index) 

In [6]:
athletics = athletics.reset_index(drop=True)

In [7]:
athletics['player_id'] = features['player_id']

In [8]:
athletics.columns

Index(['player_name', 'season', 'school_name', 'conf_abbr', 'g', 'gs', 'mp',
       'per', 'ts_pct', 'efg_pct', 'fg3a_per_fga_pct', 'fta_per_fga_pct',
       'pprod', 'orb_pct', 'drb_pct', 'trb_pct', 'ast_pct', 'stl_pct',
       'blk_pct', 'tov_pct', 'usg_pct', 'ows', 'dws', 'ws', 'ws_per_40',
       'obpm', 'dbpm', 'bpm', 'years', 'year', 'position', 'body_fat_pct',
       'hand_length', 'hand_width', 'height_wo_shoes', 'height_w_shoes',
       'standing_reach', 'weight', 'wingspan', '('Alex Len', 'weight')',
       '('Markelle Fultz', 'weight')', 'player_id'],
      dtype='object')

In [31]:
athletics.drop(athletics.columns[0:30], axis=1, inplace=True)

In [32]:
athletics

,position,body_fat_pct,hand_length,hand_width,height_wo_shoes,height_w_shoes,standing_reach,weight,wingspan,"('Alex Len', 'weight')","('Markelle Fultz', 'weight')",player_id
0,PF,7.90,229,216,2064,2096,2743,221.8,2273,0,0,anthony-davis-5
1,SF,7.00,229,260,1975,2019,2654,232.8,2134,0,0,michael-kidd-gilchrist-1
2,SG,6.00,216,229,1911,1949,2540,201.8,2032,0,0,bradley-beal-1
3,SG,8.50,216,241,1892,1930,2489,221.0,2013,0,0,dion-waiters-1
4,PF,5.00,248,267,2026,2051,2692,244.2,2216,0,0,thomas-robinson-2
...,...,...,...,...,...,...,...,...,...,...,...,...
329,PF,5.55,235,229,2057,2089,2769,215.4,2254,0,0,raymond-spalding-1
330,PF,5.15,235,260,1994,2026,2718,211.6,2222,0,0,kevin-hervey-1
331,PF,10.90,229,267,2121,2146,2781,254.0,2134,0,0,thomas-welsh-1
332,SF,7.25,229,260,1949,1981,2578,218.6,2121,0,0,george-king-1


In [33]:
all_data = ratios.merge(features, how='inner', on='player_id')

In [34]:
all_data

,player_id,player_name_x,pos,off_score,def_score,uni_off_score,uni_def_score,ratio_off,ratio_def,player_name_y,...,tov_pct,usg_pct,ows,dws,ws,ws_per_40,obpm,dbpm,bpm,years
0,anthony-davis-5,Anthony Davis,PF,3.96,3.34,3.68,3.79,1.08,0.88,Anthony Davis,...,8.6,18.8,5.9,4.1,9.9,0.310,9.1,8.1,17.2,1
1,michael-kidd-gilchrist-1,Michael Kidd-Gilchrist,SF,2.68,1.89,2.73,1.96,0.98,0.96,Michael Kidd-Gilchrist,...,17.3,20.6,3.4,2.6,6.0,0.192,4.1,3.6,7.7,1
2,bradley-beal-1,Bradley Beal,SG,2.73,1.71,2.93,1.99,0.93,0.86,Bradley Beal,...,14.3,23.0,3.6,2.1,5.7,0.180,5.4,2.9,8.4,1
3,dion-waiters-1,Dion Waiters,SG,2.36,1.49,2.97,2.04,0.79,0.73,Dion Waiters,...,10.4,26.7,3.2,1.7,5.0,0.223,6.0,4.8,10.8,2
4,thomas-robinson-2,Thomas Robinson,PF,2.68,2.17,3.03,2.67,0.88,0.81,Thomas Robinson,...,14.2,30.0,3.7,3.5,7.2,0.231,5.4,3.4,8.8,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
329,devon-hall-1,Devon Hall,NaN,NaN,NaN,2.93,1.77,NaN,NaN,Devon Hall,...,9.6,19.5,3.3,2.6,5.9,0.215,5.5,4.6,10.1,4
330,raymond-spalding-1,Raymond Spalding,NaN,NaN,NaN,2.71,2.56,NaN,NaN,Raymond Spalding,...,13.5,22.9,2.4,2.4,4.8,0.193,4.1,4.1,8.2,3
331,kevin-hervey-1,Kevin Hervey,NaN,NaN,NaN,3.05,1.66,NaN,NaN,Kevin Hervey,...,12.8,31.0,2.6,2.0,4.6,0.174,5.1,0.4,5.5,4
332,thomas-welsh-1,Thomas Welsh,NaN,NaN,NaN,2.75,1.65,NaN,NaN,Thomas Welsh,...,8.6,17.7,2.8,1.7,4.5,0.162,4.5,1.3,5.8,4


In [35]:
all_data = all_data.merge(athletics, how='inner', on='player_id')

In [36]:
all_data.drop(columns=(["('Alex Len', 'weight')","('Markelle Fultz', 'weight')"]), inplace=True)

In [37]:
all_data.columns

Index(['player_id', 'player_name_x', 'pos', 'off_score', 'def_score',
       'uni_off_score', 'uni_def_score', 'ratio_off', 'ratio_def',
       'player_name_y', 'season', 'school_name', 'conf_abbr', 'g', 'gs', 'mp',
       'per', 'ts_pct', 'efg_pct', 'fg3a_per_fga_pct', 'fta_per_fga_pct',
       'pprod', 'orb_pct', 'drb_pct', 'trb_pct', 'ast_pct', 'stl_pct',
       'blk_pct', 'tov_pct', 'usg_pct', 'ows', 'dws', 'ws', 'ws_per_40',
       'obpm', 'dbpm', 'bpm', 'years', 'position', 'body_fat_pct',
       'hand_length', 'hand_width', 'height_wo_shoes', 'height_w_shoes',
       'standing_reach', 'weight', 'wingspan'],
      dtype='object')

In [38]:
all_data.drop(columns=['bpm','ws_per_40','ws','trb_pct','pprod','conf_abbr','school_name','season','player_name_y','ratio_def','ratio_off','uni_def_score','uni_off_score','def_score','off_score','pos','player_name_x'],inplace=True)

In [45]:
all_data['gs%'] = all_data['gs']/all_data['g']

In [46]:
all_data

,player_id,g,gs,mp,per,ts_pct,efg_pct,fg3a_per_fga_pct,fta_per_fga_pct,orb_pct,...,position,body_fat_pct,hand_length,hand_width,height_wo_shoes,height_w_shoes,standing_reach,weight,wingspan,gs%
0,anthony-davis-5,40,40,1281,35.1,0.654,0.628,0.059,0.602,11.6,...,PF,7.90,229,216,2064,2096,2743,221.8,2273,1.000000
1,michael-kidd-gilchrist-1,40,39,1245,21.2,0.570,0.511,0.156,0.589,10.3,...,SF,7.00,229,260,1975,2019,2654,232.8,2134,0.975000
2,bradley-beal-1,37,37,1267,22.0,0.575,0.525,0.473,0.440,4.8,...,SG,6.00,216,229,1911,1949,2540,201.8,2032,1.000000
3,dion-waiters-1,37,0,891,26.3,0.565,0.534,0.317,0.331,2.4,...,SG,8.50,216,241,1892,1930,2489,221.0,2013,0.000000
4,thomas-robinson-2,39,39,1242,27.4,0.549,0.512,0.027,0.462,11.1,...,PF,5.00,248,267,2026,2051,2692,244.2,2216,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
329,devon-hall-1,34,34,1091,20.3,0.599,0.552,0.454,0.292,1.4,...,SG,8.45,216,254,1937,1981,2591,206.4,2026,1.000000
330,raymond-spalding-1,36,34,995,25.6,0.568,0.550,0.056,0.338,13.2,...,PF,5.55,235,229,2057,2089,2769,215.4,2254,0.944444
331,kevin-hervey-1,33,25,1064,23.7,0.553,0.517,0.417,0.257,8.0,...,PF,5.15,235,260,1994,2026,2718,211.6,2222,0.757576
332,thomas-welsh-1,33,33,1096,20.3,0.577,0.552,0.335,0.174,9.1,...,PF,10.90,229,267,2121,2146,2781,254.0,2134,1.000000


In [50]:
all_data_df = all_data.drop(columns=['player_id','gs','g','gs%'])
all_data_df

,mp,per,ts_pct,efg_pct,fg3a_per_fga_pct,fta_per_fga_pct,orb_pct,drb_pct,ast_pct,stl_pct,...,years,position,body_fat_pct,hand_length,hand_width,height_wo_shoes,height_w_shoes,standing_reach,weight,wingspan
0,1281,35.1,0.654,0.628,0.059,0.602,11.6,25.6,7.5,2.5,...,1,PF,7.90,229,216,2064,2096,2743,221.8,2273
1,1245,21.2,0.570,0.511,0.156,0.589,10.3,17.3,10.8,1.9,...,1,SF,7.00,229,260,1975,2019,2654,232.8,2134
2,1267,22.0,0.575,0.525,0.473,0.440,4.8,20.3,12.7,2.5,...,1,SG,6.00,216,229,1911,1949,2540,201.8,2032
3,891,26.3,0.565,0.534,0.317,0.331,2.4,8.8,21.2,4.6,...,2,SG,8.50,216,241,1892,1930,2489,221.0,2013
4,1242,27.4,0.549,0.512,0.027,0.462,11.1,33.0,13.1,2.0,...,3,PF,5.00,248,267,2026,2051,2692,244.2,2216
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
329,1091,20.3,0.599,0.552,0.454,0.292,1.4,14.9,19.6,1.8,...,4,SG,8.45,216,254,1937,1981,2591,206.4,2026
330,995,25.6,0.568,0.550,0.056,0.338,13.2,20.9,9.6,3.1,...,3,PF,5.55,235,229,2057,2089,2769,215.4,2254
331,1064,23.7,0.553,0.517,0.417,0.257,8.0,19.9,15.2,2.1,...,4,PF,5.15,235,260,1994,2026,2718,211.6,2222
332,1096,20.3,0.577,0.552,0.335,0.174,9.1,25.8,7.8,1.1,...,4,PF,10.90,229,267,2121,2146,2781,254.0,2134


In [51]:
all_data_df['3rd_NBA_season'] = ratios['ratio_off']

In [52]:
third_season = [value for value in all_data_df['3rd_NBA_season'] >= 0]
res = [i for i, val in enumerate(third_season) if val]
res

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,


In [53]:
a = all_data_df['3rd_NBA_season']
all_data_df['3rd_NBA_season'] = a.where(a > 0, other= 0)

In [54]:
all_data_df

,mp,per,ts_pct,efg_pct,fg3a_per_fga_pct,fta_per_fga_pct,orb_pct,drb_pct,ast_pct,stl_pct,...,position,body_fat_pct,hand_length,hand_width,height_wo_shoes,height_w_shoes,standing_reach,weight,wingspan,3rd_NBA_season
0,1281,35.1,0.654,0.628,0.059,0.602,11.6,25.6,7.5,2.5,...,PF,7.90,229,216,2064,2096,2743,221.8,2273,1.08
1,1245,21.2,0.570,0.511,0.156,0.589,10.3,17.3,10.8,1.9,...,SF,7.00,229,260,1975,2019,2654,232.8,2134,0.98
2,1267,22.0,0.575,0.525,0.473,0.440,4.8,20.3,12.7,2.5,...,SG,6.00,216,229,1911,1949,2540,201.8,2032,0.93
3,891,26.3,0.565,0.534,0.317,0.331,2.4,8.8,21.2,4.6,...,SG,8.50,216,241,1892,1930,2489,221.0,2013,0.79
4,1242,27.4,0.549,0.512,0.027,0.462,11.1,33.0,13.1,2.0,...,PF,5.00,248,267,2026,2051,2692,244.2,2216,0.88
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
329,1091,20.3,0.599,0.552,0.454,0.292,1.4,14.9,19.6,1.8,...,SG,8.45,216,254,1937,1981,2591,206.4,2026,0.00
330,995,25.6,0.568,0.550,0.056,0.338,13.2,20.9,9.6,3.1,...,PF,5.55,235,229,2057,2089,2769,215.4,2254,0.00
331,1064,23.7,0.553,0.517,0.417,0.257,8.0,19.9,15.2,2.1,...,PF,5.15,235,260,1994,2026,2718,211.6,2222,0.00
332,1096,20.3,0.577,0.552,0.335,0.174,9.1,25.8,7.8,1.1,...,PF,10.90,229,267,2121,2146,2781,254.0,2134,0.00


In [55]:
all_data_df['3rd_NBA_season'] = a.where(a <= 0, other= 1)

In [56]:
all_data_df

,mp,per,ts_pct,efg_pct,fg3a_per_fga_pct,fta_per_fga_pct,orb_pct,drb_pct,ast_pct,stl_pct,...,position,body_fat_pct,hand_length,hand_width,height_wo_shoes,height_w_shoes,standing_reach,weight,wingspan,3rd_NBA_season
0,1281,35.1,0.654,0.628,0.059,0.602,11.6,25.6,7.5,2.5,...,PF,7.90,229,216,2064,2096,2743,221.8,2273,1.0
1,1245,21.2,0.570,0.511,0.156,0.589,10.3,17.3,10.8,1.9,...,SF,7.00,229,260,1975,2019,2654,232.8,2134,1.0
2,1267,22.0,0.575,0.525,0.473,0.440,4.8,20.3,12.7,2.5,...,SG,6.00,216,229,1911,1949,2540,201.8,2032,1.0
3,891,26.3,0.565,0.534,0.317,0.331,2.4,8.8,21.2,4.6,...,SG,8.50,216,241,1892,1930,2489,221.0,2013,1.0
4,1242,27.4,0.549,0.512,0.027,0.462,11.1,33.0,13.1,2.0,...,PF,5.00,248,267,2026,2051,2692,244.2,2216,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
329,1091,20.3,0.599,0.552,0.454,0.292,1.4,14.9,19.6,1.8,...,SG,8.45,216,254,1937,1981,2591,206.4,2026,0.0
330,995,25.6,0.568,0.550,0.056,0.338,13.2,20.9,9.6,3.1,...,PF,5.55,235,229,2057,2089,2769,215.4,2254,0.0
331,1064,23.7,0.553,0.517,0.417,0.257,8.0,19.9,15.2,2.1,...,PF,5.15,235,260,1994,2026,2718,211.6,2222,0.0
332,1096,20.3,0.577,0.552,0.335,0.174,9.1,25.8,7.8,1.1,...,PF,10.90,229,267,2121,2146,2781,254.0,2134,0.0


In [22]:
all_data.columns

Index(['player_id', 'player_name_x', 'pos', 'off_score', 'def_score',
       'uni_off_score', 'uni_def_score', 'ratio_off', 'ratio_def',
       'player_name_y', 'season_x', 'school_name_x', 'conf_abbr_x', 'g_x',
       'gs_x', 'mp_x', 'per_x', 'ts_pct_x', 'efg_pct_x', 'fg3a_per_fga_pct_x',
       'fta_per_fga_pct_x', 'pprod_x', 'orb_pct_x', 'drb_pct_x', 'trb_pct_x',
       'ast_pct_x', 'stl_pct_x', 'blk_pct_x', 'tov_pct_x', 'usg_pct_x',
       'ows_x', 'dws_x', 'ws_x', 'ws_per_40_x', 'obpm_x', 'dbpm_x', 'bpm_x',
       'years_x', 'player_name', 'season_y', 'school_name_y', 'conf_abbr_y',
       'g_y', 'gs_y', 'mp_y', 'per_y', 'ts_pct_y', 'efg_pct_y',
       'fg3a_per_fga_pct_y', 'fta_per_fga_pct_y', 'pprod_y', 'orb_pct_y',
       'drb_pct_y', 'trb_pct_y', 'ast_pct_y', 'stl_pct_y', 'blk_pct_y',
       'tov_pct_y', 'usg_pct_y', 'ows_y', 'dws_y', 'ws_y', 'ws_per_40_y',
       'obpm_y', 'dbpm_y', 'bpm_y', 'years_y', 'year', 'position',
       'body_fat_pct', 'hand_length', 'hand_width', 'h

In [57]:
all_data_df['3rd_NBA_season'] = [int(i) for i in all_data_df['3rd_NBA_season']]

In [58]:
all_data_df

,mp,per,ts_pct,efg_pct,fg3a_per_fga_pct,fta_per_fga_pct,orb_pct,drb_pct,ast_pct,stl_pct,...,position,body_fat_pct,hand_length,hand_width,height_wo_shoes,height_w_shoes,standing_reach,weight,wingspan,3rd_NBA_season
0,1281,35.1,0.654,0.628,0.059,0.602,11.6,25.6,7.5,2.5,...,PF,7.90,229,216,2064,2096,2743,221.8,2273,1
1,1245,21.2,0.570,0.511,0.156,0.589,10.3,17.3,10.8,1.9,...,SF,7.00,229,260,1975,2019,2654,232.8,2134,1
2,1267,22.0,0.575,0.525,0.473,0.440,4.8,20.3,12.7,2.5,...,SG,6.00,216,229,1911,1949,2540,201.8,2032,1
3,891,26.3,0.565,0.534,0.317,0.331,2.4,8.8,21.2,4.6,...,SG,8.50,216,241,1892,1930,2489,221.0,2013,1
4,1242,27.4,0.549,0.512,0.027,0.462,11.1,33.0,13.1,2.0,...,PF,5.00,248,267,2026,2051,2692,244.2,2216,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
329,1091,20.3,0.599,0.552,0.454,0.292,1.4,14.9,19.6,1.8,...,SG,8.45,216,254,1937,1981,2591,206.4,2026,0
330,995,25.6,0.568,0.550,0.056,0.338,13.2,20.9,9.6,3.1,...,PF,5.55,235,229,2057,2089,2769,215.4,2254,0
331,1064,23.7,0.553,0.517,0.417,0.257,8.0,19.9,15.2,2.1,...,PF,5.15,235,260,1994,2026,2718,211.6,2222,0
332,1096,20.3,0.577,0.552,0.335,0.174,9.1,25.8,7.8,1.1,...,PF,10.90,229,267,2121,2146,2781,254.0,2134,0


In [29]:
scaler = RobustScaler()
mm_scaler = MinMaxScaler()

In [59]:
X = all_data_df[['mp','per','ts_pct','efg_pct','fg3a_per_fga_pct','fta_per_fga_pct',
            'orb_pct','drb_pct','ast_pct','stl_pct','blk_pct','tov_pct','usg_pct','ows','dws','obpm','dbpm','years']]
y = all_data_df[['3rd_NBA_season']]

In [60]:
X_scaled = scaler.fit_transform(X)

In [61]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled,y, train_size=0.8)

In [62]:
model = LogisticRegression(max_iter=300)
cv_results = cross_validate(model, X_train, y_train, cv=5, scoring=['precision'])

/Users/olivierpavee/.pyenv/versions/3.8.6/envs/nba_forecast/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/Users/olivierpavee/.pyenv/versions/3.8.6/envs/nba_forecast/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/Users/olivierpavee/.pyenv/versions/3.8.6/envs/nba_forecast/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/Users/olivierpavee/.pyenv/versions/3.8.6/envs/nba_forecast/lib/python3.8/site-packages/

In [63]:
cv_results

{'fit_time': array([0.01867986, 0.01450419, 0.0145247 , 0.01051497, 0.01306987]),
 'score_time': array([0.00203824, 0.00248194, 0.0013752 , 0.00141788, 0.00175929]),
 'test_precision': array([0.70212766, 0.71698113, 0.76086957, 0.79166667, 0.74509804])}

In [64]:
base_score = cv_results['test_precision'].mean()
base_score

0.7433486125499369

In [65]:
y_train = y_train.squeeze()
y_train

278    0
299    0
229    1
244    1
313    0
      ..
9      1
148    1
173    1
255    0
129    1
Name: 3rd_NBA_season, Length: 267, dtype: int64

In [66]:
proba_0, proba_1 = cross_val_predict(LogisticRegression(),X_train, y_train.to_list(),method = "predict_proba").T

In [67]:
precision, recall, thresholds = precision_recall_curve(y_train, proba_1)

In [77]:
df_precision = pd.DataFrame({"precision" : precision[:-1], "threshold" : thresholds})
new_threshold = df_precision[df_precision['precision'] >= 0.75]['threshold'].min()
new_threshold

0.5203801326050291

In [81]:
model = LogisticRegression()
model.fit(X_train, y_train)

def custom_predict(X_train, custom_threshold):
    probs = model.predict_proba(X_train)
    return probs

custom_prediction = custom_predict(X_test, custom_threshold=new_threshold)
predictions = pd.DataFrame(custom_prediction, columns=['0','1'])

In [82]:
predictions

,0,1
0,0.654782,0.345218
1,0.127916,0.872084
2,0.363765,0.636235
3,0.161206,0.838794
4,0.267923,0.732077
...,...,...
62,0.235253,0.764747
63,0.321437,0.678563
64,0.201554,0.798446
65,0.367305,0.632695
